# 第7章: お問い合わせフォームを作ろう

## 学習目標
- HTMLフォームとPHPの連携を理解する
- POSTリクエストとGETリクエストの違いを学ぶ
- フォームデータの受け取り方をマスターする
- セキュリティ対策（バリデーション、サニタイズ）を理解する
- 実用的なお問い合わせフォームを作成できる

---
## 1. フォームの基本

フォームはユーザーからデータを受け取るためのUIです。

### 🔍 フォーム処理の流れ

```
ユーザー入力
    ↓
HTMLフォーム
    ↓
送信ボタンクリック
    ↓
PHPでデータ受取
    ↓
バリデーション
    ↓
処理・保存
    ↓
結果表示
```

---
## 2. 基本的なフォームの作成

### HTMLフォーム（form.html）

In [ ]:
<!-- form.html -->
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <title>お問い合わせフォーム</title>
    <style>
        .form-group {
            margin-bottom: 15px;
        }
        label {
            display: block;
            margin-bottom: 5px;
            font-weight: bold;
        }
        input[type="text"],
        input[type="email"],
        textarea {
            width: 100%;
            padding: 8px;
            border: 1px solid #ddd;
            border-radius: 4px;
            box-sizing: border-box;
        }
        textarea {
            height: 150px;
        }
        button {
            background: #007bff;
            color: white;
            border: none;
            padding: 10px 20px;
            border-radius: 4px;
            cursor: pointer;
        }
        button:hover {
            background: #0056b3;
        }
        .error {
            color: red;
            font-size: 0.9em;
        }
    </style>
</head>
<body>
    <h1>お問い合わせ</h1>
    <form action="process.php" method="POST">
        <div class="form-group">
            <label for="name">お名前 <span style="color:red">*</span></label>
            <input type="text" id="name" name="name" required>
        </div>
        
        <div class="form-group">
            <label for="email">メールアドレス <span style="color:red">*</span></label>
            <input type="email" id="email" name="email" required>
        </div>
        
        <div class="form-group">
            <label for="subject">件名</label>
            <input type="text" id="subject" name="subject">
        </div>
        
        <div class="form-group">
            <label for="message">お問い合わせ内容 <span style="color:red">*</span></label>
            <textarea id="message" name="message" required></textarea>
        </div>
        
        <button type="submit">送信</button>
    </form>
</body>
</html>

---
## 3. POSTとGETの違い

### POSTメソッド
- **データをリクエストボディに含めて送信**
- 大容量のデータを送信できる
- ブラウザの履歴に残らない
- センシティブなデータの送信に適している

### GETメソッド
- **データをURLパラメータとして送信**
- `?key=value&key2=value2` の形式
- ブックマークや共有が可能
- 検索クエリなどに適している

| 特徴 | POST | GET |
|------|------|-----|
| データの場所 | リクエストボディ | URLパラメータ |
| データ容量 | 大容量可 | 制限あり（約2000文字） |
| セキュリティ | 高い | 低い（URLに残る） |
| ブックマーク | 不可 | 可能 |
| 用途 | フォーム送信 | 検索、ページネーション |

---
## 4. フォームデータの受け取り

### $_POSTと$_GET

PHPはスーパーグローバル変数 `$_POST`、`$_GET` でフォームデータを受け取ります。

In [ ]:
<?php
// process.php

// POSTデータの受け取り
if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    $name = $_POST['name'] ?? '';
    $email = $_POST['email'] ?? '';
    $subject = $_POST['subject'] ?? '';
    $message = $_POST['message'] ?? '';
    
    // 受け取ったデータを確認
    echo "<pre>";
    echo "お名前: " . htmlspecialchars($name, ENT_QUOTES, 'UTF-8') . "\n";
    echo "メール: " . htmlspecialchars($email, ENT_QUOTES, 'UTF-8') . "\n";
    echo "件名: " . htmlspecialchars($subject, ENT_QUOTES, 'UTF-8') . "\n";
    echo "内容: " . htmlspecialchars($message, ENT_QUOTES, 'UTF-8') . "\n";
    echo "</pre>";
}

?>

---
## 5. バリデーション（入力チェック）

ユーザー入力は必ずチェックしましょう！

In [ ]:
<?php

// バリデーション関数
function validateForm(array $data): array {
    $errors = [];
    
    // お名前：必須、1〜50文字
    if (empty($data['name'])) {
        $errors['name'] = 'お名前は必須です';
    } elseif (mb_strlen($data['name']) > 50) {
        $errors['name'] = 'お名前は50文字以内で入力してください';
    }
    
    // メールアドレス：必須、形式チェック
    if (empty($data['email'])) {
        $errors['email'] = 'メールアドレスは必須です';
    } elseif (!filter_var($data['email'], FILTER_VALIDATE_EMAIL)) {
        $errors['email'] = '正しいメールアドレスを入力してください';
    }
    
    // 件名：100文字まで
    if (!empty($data['subject']) && mb_strlen($data['subject']) > 100) {
        $errors['subject'] = '件名は100文字以内で入力してください';
    }
    
    // お問い合わせ内容：必須、10〜1000文字
    if (empty($data['message'])) {
        $errors['message'] = 'お問い合わせ内容は必須です';
    } elseif (mb_strlen($data['message']) < 10) {
        $errors['message'] = 'お問い合わせ内容は10文字以上入力してください';
    } elseif (mb_strlen($data['message']) > 1000) {
        $errors['message'] = 'お問い合わせ内容は1000文字以内で入力してください';
    }
    
    return $errors;
}

// 使用例
$formData = [
    'name' => '山田太郎',
    'email' => 'invalid-email',
    'subject' => '問い合わせ',
    'message' => '短すぎる'
];

$errors = validateForm($formData);
if (!empty($errors)) {
    echo "バリデーションエラー:\n";
    print_r($errors);
}

?>

---
## 6. サニタイズ（無害化）

XSS（クロスサイトスクリプティング）対策のために、ユーザー入力を無害化します。

In [ ]:
<?php

// サニタイズ関数
function sanitize(string $value): string {
    return htmlspecialchars($value, ENT_QUOTES, 'UTF-8');
}

// 配列をサニタイズ
function sanitizeArray(array $data): array {
    return array_map(function($value) {
        if (is_array($value)) {
            return sanitizeArray($value);
        }
        return htmlspecialchars($value, ENT_QUOTES, 'UTF-8');
    }, $data);
}

// 悪意のある入力例
$malicious = '<script>alert("XSS")</script>';

echo "元の値: " . $malicious . "\n";
echo "危険！ブラウザで実行されてしまう\n\n";

echo "サニタイズ後: " . sanitize($malicious) . "\n";
echo "安全！文字として表示される\n";

?>

---
## 7. CSRF対策

CSRF（クロスサイトリクエストフォージェリ）対策にトークンを使用します。

In [ ]:
<?php
session_start();

// CSRFトークンの生成
function generateCsrfToken(): string {
    if (!isset($_SESSION['csrf_token'])) {
        $_SESSION['csrf_token'] = bin2hex(random_bytes(32));
    }
    return $_SESSION['csrf_token'];
}

// CSRFトークンの検証
function validateCsrfToken(string $token): bool {
    return isset($_SESSION['csrf_token']) && hash_equals($_SESSION['csrf_token'], $token);
}

// フォームにトークンを埋め込む
function getCsrfField(): string {
    $token = generateCsrfToken();
    return '<input type="hidden" name="csrf_token" value="' . htmlspecialchars($token, ENT_QUOTES, 'UTF-8') . '">';
}

?>

---
## 8. 完全なお問い合わせフォーム

### contact.php（送信フォーム表示＋処理）

In [ ]:
<?php
session_start();

// バリデーション関数
function validateForm(array $data): array {
    $errors = [];
    
    if (empty($data['name']) || mb_strlen($data['name']) > 50) {
        $errors['name'] = 'お名前は必須です（50文字以内）';
    }
    
    if (empty($data['email']) || !filter_var($data['email'], FILTER_VALIDATE_EMAIL)) {
        $errors['email'] = '正しいメールアドレスを入力してください';
    }
    
    if (empty($data['message']) || mb_strlen($data['message']) < 10 || mb_strlen($data['message']) > 1000) {
        $errors['message'] = 'お問い合わせ内容は10〜1000文字で入力してください';
    }
    
    return $errors;
}

// CSRFトークン生成
function generateCsrfToken(): string {
    if (!isset($_SESSION['csrf_token'])) {
        $_SESSION['csrf_token'] = bin2hex(random_bytes(32));
    }
    return $_SESSION['csrf_token'];
}

// メール送信関数
function sendEmail(array $data): bool {
    $to = 'info@example.com';
    $subject = $data['subject'] ?? 'お問い合わせ';
    $subject = '=?UTF-8?B?' . base64_encode($subject) . '?=';
    
    $message = "お名前: {$data['name']}\n";
    $message .= "メール: {$data['email']}\n";
    $message .= "\nお問い合わせ内容:\n{$data['message']}";
    
    $headers = "From: {$data['email']}\n";
    $headers .= "Reply-To: {$data['email']}\n";
    $headers .= "Content-Type: text/plain; charset=UTF-8\n";
    
    return mail($to, $subject, $message, $headers);
}

// 変数初期化
$errors = [];
$success = false;
$formData = [
    'name' => '',
    'email' => '',
    'subject' => '',
    'message' => ''
];

// フォーム送信時の処理
if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    // CSRFトークン検証
    if (!isset($_POST['csrf_token']) || !hash_equals($_SESSION['csrf_token'], $_POST['csrf_token'])) {
        $errors['csrf'] = '不正なリクエストです';
    } else {
        // フォームデータ取得
        $formData = [
            'name' => trim($_POST['name'] ?? ''),
            'email' => trim($_POST['email'] ?? ''),
            'subject' => trim($_POST['subject'] ?? ''),
            'message' => trim($_POST['message'] ?? '')
        ];
        
        // バリデーション
        $errors = validateForm($formData);
        
        // エラーがなければ送信
        if (empty($errors)) {
            if (sendEmail($formData)) {
                $success = true;
                // 送信成功後、フォームデータをリセット
                $formData = array_fill(0, 4, '');
            } else {
                $errors['send'] = '送信に失敗しました。時間をおいて再度お試しください。';
            }
        }
    }
}

$csrfToken = generateCsrfToken();

?>
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>お問い合わせ</title>
    <style>
        * { box-sizing: border-box; }
        body {
            font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;
            line-height: 1.6;
            max-width: 600px;
            margin: 50px auto;
            padding: 20px;
        }
        .form-group { margin-bottom: 20px; }
        label { display: block; margin-bottom: 5px; font-weight: bold; }
        input, textarea {
            width: 100%;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
            font-size: 16px;
        }
        textarea { height: 150px; resize: vertical; }
        button {
            background: #007bff;
            color: white;
            border: none;
            padding: 12px 24px;
            border-radius: 4px;
            font-size: 16px;
            cursor: pointer;
        }
        button:hover { background: #0056b3; }
        .error { color: #dc3545; font-size: 14px; margin-top: 5px; }
        .success {
            background: #d4edda;
            color: #155724;
            padding: 15px;
            border-radius: 4px;
            margin-bottom: 20px;
        }
        .danger {
            background: #f8d7da;
            color: #721c24;
            padding: 15px;
            border-radius: 4px;
            margin-bottom: 20px;
        }
    </style>
</head>
<body>
    <h1>お問い合わせフォーム</h1>
    
    <?php if ($success): ?>
        <div class="success">
            お問い合わせありがとうございます。<br>
            内容を確認次第、ご連絡いたします。
        </div>
    <?php endif; ?>
    
    <?php if (isset($errors['csrf'])): ?>
        <div class="danger"><?php echo htmlspecialchars($errors['csrf'], ENT_QUOTES, 'UTF-8'); ?></div>
    <?php endif; ?>
    
    <?php if (isset($errors['send'])): ?>
        <div class="danger"><?php echo htmlspecialchars($errors['send'], ENT_QUOTES, 'UTF-8'); ?></div>
    <?php endif; ?>
    
    <form method="POST">
        <input type="hidden" name="csrf_token" value="<?php echo htmlspecialchars($csrfToken, ENT_QUOTES, 'UTF-8'); ?>">
        
        <div class="form-group">
            <label for="name">お名前 <span style="color:red">*</span></label>
            <input type="text" id="name" name="name" 
                   value="<?php echo htmlspecialchars($formData['name'], ENT_QUOTES, 'UTF-8'); ?>" required>
            <?php if (isset($errors['name'])): ?>
                <div class="error"><?php echo htmlspecialchars($errors['name'], ENT_QUOTES, 'UTF-8'); ?></div>
            <?php endif; ?>
        </div>
        
        <div class="form-group">
            <label for="email">メールアドレス <span style="color:red">*</span></label>
            <input type="email" id="email" name="email" 
                   value="<?php echo htmlspecialchars($formData['email'], ENT_QUOTES, 'UTF-8'); ?>" required>
            <?php if (isset($errors['email'])): ?>
                <div class="error"><?php echo htmlspecialchars($errors['email'], ENT_QUOTES, 'UTF-8'); ?></div>
            <?php endif; ?>
        </div>
        
        <div class="form-group">
            <label for="subject">件名</label>
            <input type="text" id="subject" name="subject" 
                   value="<?php echo htmlspecialchars($formData['subject'], ENT_QUOTES, 'UTF-8'); ?>">
        </div>
        
        <div class="form-group">
            <label for="message">お問い合わせ内容 <span style="color:red">*</span></label>
            <textarea id="message" name="message" required><?php echo htmlspecialchars($formData['message'], ENT_QUOTES, 'UTF-8'); ?></textarea>
            <?php if (isset($errors['message'])): ?>
                <div class="error"><?php echo htmlspecialchars($errors['message'], ENT_QUOTES, 'UTF-8'); ?></div>
            <?php endif; ?>
        </div>
        
        <button type="submit">送信</button>
    </form>
</body>
</html>

---
## 9. ファイルアップロード

ファイルをアップロードするフォームの作り方です。

In [ ]:
<?php
session_start();

$uploadDir = __DIR__ . '/uploads/';
$maxFileSize = 5 * 1024 * 1024; // 5MB
$allowedTypes = ['image/jpeg', 'image/png', 'image/gif', 'application/pdf'];

// アップロードディレクトリがなければ作成
if (!file_exists($uploadDir)) {
    mkdir($uploadDir, 0755, true);
}

$message = '';
$error = '';

if ($_SERVER['REQUEST_METHOD'] === 'POST' && isset($_FILES['file'])) {
    $file = $_FILES['file'];
    
    // エラーチェック
    if ($file['error'] !== UPLOAD_ERR_OK) {
        $error = 'アップロードに失敗しました';
    }
    // ファイルサイズチェック
    elseif ($file['size'] > $maxFileSize) {
        $error = 'ファイルサイズは5MB以下にしてください';
    }
    // ファイルタイプチェック
    elseif (!in_array($file['type'], $allowedTypes)) {
        $error = '許可されていないファイル形式です';
    }
    else {
        // ファイル名のサニタイズ
        $filename = basename($file['name']);
        $filename = preg_replace('/[^A-Za-z0-9_\-\.]/', '_', $filename);
        $filename = time() . '_' . $filename; // タイムスタンプを付けて重複回避
        
        $uploadFile = $uploadDir . $filename;
        
        if (move_uploaded_file($file['tmp_name'], $uploadFile)) {
            $message = 'ファイルが正常にアップロードされました';
        } else {
            $error = 'ファイルの保存に失敗しました';
        }
    }
}

?>
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <title>ファイルアップロード</title>
</head>
<body>
    <h1>ファイルアップロード</h1>
    
    <?php if ($message): ?>
        <p style="color: green"><?php echo htmlspecialchars($message, ENT_QUOTES, 'UTF-8'); ?></p>
    <?php endif; ?>
    
    <?php if ($error): ?>
        <p style="color: red"><?php echo htmlspecialchars($error, ENT_QUOTES, 'UTF-8'); ?></p>
    <?php endif; ?>
    
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="file" required>
        <button type="submit">アップロード</button>
    </form>
    
    <p>許可されているファイル: JPEG, PNG, GIF, PDF（最大5MB）</p>
</body>
</html>

---
## 📝 実践問題

### 問題1: アンケートフォーム

以下の項目を持つアンケートフォームを作成してください。

- 満足度（ラジオボタン: 非常に不満〜非常に満足）
- 改善点（テキストエリア）
- 再利用意向（チェックボックス）

### 問題2: 検索フォーム

GETメソッドを使った検索フォームを作成してください。

- 検索ワード入力
- カテゴリ選択（セレクトボックス）
- 結果表示ページへの遷移

### 問題3: 多言語対応

お問い合わせフォームを英語と日本語に対応させてください。

### 問題4: 入力内容の確認画面

お問い合わせフォームに確認画面を追加してください。

入力 → 確認 → 完了 の流れ

---
## まとめ

### ✅ ポイントチェック

- [ ] POSTとGETの違いを理解した
- [ ] フォームデータの受け取り方がわかった
- [ ] バリデーションの実装ができる
- [ ] サニタイズの重要性を理解した
- [ ] CSRF対策ができるようになった
- [ ] ファイルアップロードの実装ができる

### 🔒 セキュリティチェックリスト

- [ ] 全てのユーザー入力をバリデートしている
- [ ] HTML出力時にサニタイズしている
- [ ] CSRFトークンを使用している
- [ ] ファイルアップロードの検証をしている
- [ ] エラーメッセージで詳細な情報を出しすぎていない

### 🚀 次のステップ

次は「まとめ（1〜7章の総復習）」で、これまで学んだ内容を復習しましょう！

---

**関連リソース**
- [PHP: フォーム処理](https://www.php.net/manual/ja/tutorial.forms.php)
- [OWASP: XSS Prevention](https://owasp.org/www-community/attacks/xss/)
- [OWASP: CSRF Prevention](https://owasp.org/www-community/attacks/csrf)